In [70]:
# IMPORT THE LIBRARY
import yfinance as yf
from datetime import datetime
import pandas as pd
import pickle
import numpy as np
import math



In [71]:
with open('histdf.pickle', 'rb') as handle:
    histdata = pickle.load(handle)
rollingdays=10
backdays=10

# Shift go back N days not including current date
# Rolling window go back N days including today

etfs=list(histdata.keys())
noetf=['IPK', 'IST', 'IPW', 'IPS']
etfs= list(set(etfs)-set(noetf))
for etf in etfs:
    df=histdata[etf]
    df['dailyreturns'] = df['Close'].pct_change()
    df['volatility'] = df['dailyreturns'].rolling(window=rollingdays).std()
    df['pre1volatility'] = df['volatility'].shift(backdays)
    
    
    df['avgreturns'] = df['dailyreturns'].rolling(window=rollingdays).mean()
    df['pre1avgreturns'] = df['avgreturns'].shift(backdays)
    df['prev1ExpectedLoss']=df['pre1avgreturns']-1.5*df['pre1volatility']* backdays**(0.5)
    df['pre2avgreturns'] = df['avgreturns'].shift(backdays*2)


    df["prev1Close"]= df.Close.shift(backdays)
    df["prev2Close"]= df.Close.shift(backdays*2)

    df['HighMax']=df['High'].rolling(window=backdays).max() # This includes current date but not the date when trade was made
    df['LowMin']=df['Low'].rolling(window=backdays).min()
    
    

    
    df['avgClose1'] = df['Close'].rolling(window=rollingdays*1).mean()
    df['avgClose2'] = df['Close'].rolling(window=rollingdays*2).mean()
    df["ind"]= np.where((df['Close'] >df['avgClose1'] ) &  (df['avgClose1'] >df['avgClose2'] ),1,0)
    df['pre1avgClose'] = df['avgClose1'].shift(backdays)
    df['pre2avgClose'] = df['avgClose1'].shift(backdays*2)

    df['ExpectedGain']= (df['prev1Close']/df['prev2Close']-1) #- df['pre1volatility']* backdays**(0.5)
    
    


    df['pctchange']=df['Close']/df["prev1Close"]-1
    
    #df['maxpctchange'] =  np.where(df['LowMin']/df["prev1Close"]<(1+df['prev1ExpectedLoss']),df['prev1ExpectedLoss'],df['pctchange'])
    #df['maxpctchange'] =  df['Close']/df["prev1Close"]-1
    #df['maxpctchange'] =  np.where(df['HighMax']/df["prev1Close"]>(1+df['ExpectedGain']),df['ExpectedGain'],df['pctchange'])


    
    
    df['prev1MoveAvgchange']=df["ind"]*(df['prev1Close']/df['pre1avgClose']-1) #/df['pre1volatility']

    df['maxpctchange'] =  np.where((df['HighMax']/df["prev1Close"]-1)>(df['prev1MoveAvgchange']/2),(df['prev1MoveAvgchange']/2),df['pctchange'])

    histdata[etf]=df


In [72]:


df= histdata["SPY"]
df.to_csv("testspyall.csv")
alldates= df.index.tolist()
dates=alldates[0::backdays]
df=df.loc[dates]
#df.fillna(0, inplace=True)
df.to_csv("testspybackdays.csv")
tickvals={}
for dt in dates:
    try:   
        #tickvals[dt]=[( df.loc[dt]['Ticker']', df.loc[dt][what],df.loc[dt][what],df.loc[dt]['prevpctchange'])]
        tickvals[dt]=[('Dummy', -100,-100)]
    except:
        pass


In [73]:
when='maxpctchange' 
what='prev1MoveAvgchange' #'prevpctchange' #'preavgreturns'#'prevpctvol' #'preavgreturns'
for etf in etfs:
    df=histdata[etf]
    datesdf= list(set(dates).intersection(set(df.index.tolist())))
    df=df.loc[datesdf]
    df.fillna(0, inplace=True)      
    for dt in dates:
        try:    
            tickvals[dt]=tickvals[dt]+[( df.loc[dt]['Ticker'], df.loc[dt][when],df.loc[dt][what])]
        except :
            pass
with open('tickvals.pickle', 'wb') as handle:
    pickle.dump(tickvals, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [89]:
dtrng=dates[4:]
print(dtrng)

[Timestamp('2013-03-01 00:00:00'), Timestamp('2013-03-15 00:00:00'), Timestamp('2013-04-01 00:00:00'), Timestamp('2013-04-15 00:00:00'), Timestamp('2013-04-29 00:00:00'), Timestamp('2013-05-13 00:00:00'), Timestamp('2013-05-28 00:00:00'), Timestamp('2013-06-11 00:00:00'), Timestamp('2013-06-25 00:00:00'), Timestamp('2013-07-10 00:00:00'), Timestamp('2013-07-24 00:00:00'), Timestamp('2013-08-07 00:00:00'), Timestamp('2013-08-21 00:00:00'), Timestamp('2013-09-05 00:00:00'), Timestamp('2013-09-19 00:00:00'), Timestamp('2013-10-03 00:00:00'), Timestamp('2013-10-17 00:00:00'), Timestamp('2013-10-31 00:00:00'), Timestamp('2013-11-14 00:00:00'), Timestamp('2013-11-29 00:00:00'), Timestamp('2013-12-13 00:00:00'), Timestamp('2013-12-30 00:00:00'), Timestamp('2014-01-14 00:00:00'), Timestamp('2014-01-29 00:00:00'), Timestamp('2014-02-12 00:00:00'), Timestamp('2014-02-27 00:00:00'), Timestamp('2014-03-13 00:00:00'), Timestamp('2014-03-27 00:00:00'), Timestamp('2014-04-10 00:00:00'), Timestamp('20

In [92]:
N=5
amt=50000
spyval=amt

#dtrng=dates[4:]
spydata=histdata["SPY"].loc[dtrng]
spyval= spydata.tail(1)['Close'][0]/spydata.head(1)['Close'][0]*spyval



with open('tickvals.pickle', 'rb') as handle:
    tickvals = pickle.load(handle)

items={}
dtnotworking=[] # if (len(dtnotworking)==0): [] else dtnotworking

for dt in dtrng:            
  
    dict_list = {t[2]: t for t in tickvals[dt]}      
    
    # getting the keys in descending order
    keys = sorted(dict_list.keys(), reverse=True)

    # getting the top N tuples
    res = []
    vl=0
    for i in range(N):
        try:
            res.append(dict_list[keys[i]])                
            vl=vl+dict_list[keys[i]][1] 
        except Exception as e:
            print(dt)
            print(e)                      
    
    amt=amt+(amt*vl/N)
     
    
    items[dt]=[a for a, b, v in res]+['{:.2f}%'.format(b*100) for a, b,v in res]+ ["gap"]+['{:.2f}%'.format(v*100) for a, b, v in res]+[amt]+[vl]
    

print("S&P Buy and hold return :", '{:.2f}'.format(spyval) , "This Strategy :", '{:.2f}'.format(amt))


S&P Buy and hold return : 175445.09 This Strategy : 5084080.21


In [88]:
pd.DataFrame.from_dict(items, orient='index').to_csv("test.csv")